In [24]:
import matplotlib.pyplot as plt

from keras.layers.advanced_activations import LeakyReLU
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras import datasets, layers, models

from keras.optimizers import SGD
import h5py as h5
import os

import scipy.io
from scipy.io import loadmat
import cv2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics
import numpy as np




In [25]:
tf.__version__

'1.14.0'

Data fetching and understand the train/val/test splits.  

In [26]:
# Open the file as readonly
h5f = h5.File('SVHN_single_grey1.h5', 'r')

# Load the training, test and validation set
X_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
X_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]
X_val = h5f['X_val'][:]
y_val = h5f['y_val'][:]

In [27]:
X_train.shape

(42000, 32, 32)

In [28]:
X_test.shape

(18000, 32, 32)

In [29]:
y_train.shape

(42000,)

In [30]:
X_train.size

43008000

In [31]:
X_train = np.reshape(X_train, (-1,1024))
X_test = np.reshape(X_test, (-1,1024 ))

In [32]:
X_train.shape

(42000, 1024)

Implement and apply an optimal k-Nearest Neighbor (kNN) classifier 

In [10]:
NNH = KNeighborsClassifier(n_neighbors= 9)
NNH.fit(X_train,y_train)
ypred = NNH.predict(X_test)

Print the classification metric report

In [11]:
accuracy=metrics.accuracy_score(y_test,ypred)
print("The accuracy when is {}".format(accuracy))

The accuracy when is 0.5124444444444445


In [12]:
neighbors = [2,3,7,9]
for x in neighbors:
    NNH = KNeighborsClassifier(n_neighbors= x)
    NNH.fit(X_train,y_train)
    ypred = NNH.predict(X_test)
    accuracy=metrics.accuracy_score(y_test,ypred)
    print("The accuracy when k={} is {}".format(x,accuracy))

The accuracy when k=2 is 0.43644444444444447
The accuracy when k=3 is 0.4617777777777778
The accuracy when k=7 is 0.5070555555555556
The accuracy when k=9 is 0.5124444444444445


In [13]:
 # Close this file
h5f.close()
print('Training set', X_train.shape, y_train.shape)
print('Validation set', X_val.shape, y_val.shape)
print('Test set', X_test.shape, y_test.shape)

Training set (42000, 1024) (42000,)
Validation set (60000, 32, 32) (60000,)
Test set (18000, 1024) (18000,)


In [33]:
# Open the file as readonly
h5f = h5.File('SVHN_single_grey1.h5', 'r')

# Load the training, test and validation set
X_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
X_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]
X_val = h5f['X_val'][:]
y_val = h5f['y_val'][:]

In [34]:
X_train = X_train.reshape((42000, 32, 32, 1))
X_test = X_test.reshape((18000, 32, 32, 1))

In [35]:
#Early stopping with patience=5
es = EarlyStopping(monitor='val_acc', mode='max', patience=5)

Implement and apply a deep neural network classifier including and Batch Normalization for Neural Network

In [36]:
#Initialize model
model = tf.keras.models.Sequential()
#Add first convolutional layer
model.add(tf.keras.layers.Conv2D(32, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu', input_shape=(32,32,1)))
model.add(tf.keras.layers.BatchNormalization())
#Add second convolutional layer
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
#model.output
#Flatten the output
model.add(tf.keras.layers.Flatten())
#Dense layer
model.add(tf.keras.layers.Dense(128, activation='relu'))
#Output layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               3211392   
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [38]:
#Train the model
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=10,
          batch_size=32, 
          callbacks=[es])

Train on 42000 samples, validate on 18000 samples
Epoch 1/10
42000/42000 [==============================] - 34s 809us/sample - loss: 1.1683 - acc: 0.7400 - val_loss: 0.5819 - val_acc: 0.8268
Epoch 2/10
42000/42000 [==============================] - 32s 773us/sample - loss: 0.4319 - acc: 0.8690 - val_loss: 0.4877 - val_acc: 0.8584
Epoch 3/10
42000/42000 [==============================] - 32s 772us/sample - loss: 0.3237 - acc: 0.8996 - val_loss: 0.5740 - val_acc: 0.8497
Epoch 4/10
42000/42000 [==============================] - 33s 776us/sample - loss: 0.2497 - acc: 0.9217 - val_loss: 0.8731 - val_acc: 0.7708
Epoch 5/10
42000/42000 [==============================] - 33s 780us/sample - loss: 0.1984 - acc: 0.9380 - val_loss: 0.5875 - val_acc: 0.8652
Epoch 6/10
42000/42000 [==============================] - 33s 774us/sample - loss: 0.1625 - acc: 0.9487 - val_loss: 0.6448 - val_acc: 0.8696
Epoch 7/10
42000/42000 [==============================] - 33s 779us/sample - loss: 0.1298 - acc: 0.9593 

Understand the differences and trade-offs between traditional and NN classifiers with the help of classification metrics

In [39]:
model.evaluate(X_test,y_test)

18000/18000 [==============================] - 4s 210us/sample - loss: 1.0299 - acc: 0.8512


[1.029925641781754, 0.85116667]

In [ ]:
#It is observed that the accuracy has increased from 51.24 in case of K Neighbors classifier to 85.12 in case of neural network.
#This is quite a significant progress.